#Get Authorization Token for the API

In [ ]:
import requests

baseUrl = 'https://auth.cloud.esa-worldcereal.org'

data = {
    'username': 'demo_rdm',
    'password': 'rdmlpsdemo2022',
    'client_id': 'auth',
    'grant_type': 'password',
}

tokenResponse = requests.post(f'{baseUrl}/realms/worldcereal/protocol/openid-connect/token', data=data)

token = tokenResponse.json()

token

#Form Authentication Headers

In [ ]:
apiUrl = 'https://rdmapi.cloud.esa-worldcereal.org/data'
bearer = token['token_type']
accessToken = token['access_token']
headers = {
    'Authorization': f'{bearer} {accessToken}',
}

headers

#Get All Public Collections

In [ ]:
collectionResponse = requests.get(f'{apiUrl}/collections', headers=headers)
collectionResponse.json()

#Get Features from collection

In [ ]:
collectionId = collectionResponse.json()['items'][0]['collectionId']

itemUrl = f'{apiUrl}/collections/{collectionId}/items'

itemsResponse = requests.get(itemUrl, headers=headers)
itemsResponse.json()